In [13]:
from scipy.stats import mannwhitneyu, normaltest
import pandas as pd
import numpy as np
from math import sqrt
from matplotlib import pyplot as plt
from typing import Tuple, List
from statistics import mean
import os

%matplotlib inline
%cd ~/github/geneticProgramming/seminar/src

/Users/rmn/github/geneticProgramming/seminar/src


In [14]:
# save all results to PATH (=rmd project folder)

PATH = "../docs/rmd/"
TABLE_PATH = f"{PATH}/tables"
PLOT_PATH =  f"{PATH}/plots"

jobs = (
    'os.makedirs(f"{TABLE_PATH}/csv")',
    'os.makedirs(f"{TABLE_PATH}/md")',
    'os.makedirs(f"{PLOT_PATH}")'
)

for job in jobs:
    try:
        exec(job)
    except FileExistsError:
        pass
    


In [15]:
def tsv_to_df(algorithm: str, id: int) -> pd.DataFrame:
    """Return the results for ../results/single_run/<algorithm><id>.tsv as a pd.DataFrame"""
    
    path = f"../results/single_run/{algorithm}/{id}.tsv"
    
    df = pd.read_csv(filepath_or_buffer=path, sep="\t", index_col=False, skipinitialspace=True)
    
    df.columns = df.columns.str.strip()
    
    rename_dict = {
        "avg" : "mean_training_se",
        "std" : "std_training_se",
        "min" : "min_training_se",
        "max" : "max_training_se",
    }
        
    return df.rename(columns=rename_dict) 


# read and store all log files into dataframes
tournament_logs = []
elexicase_logs = []

for n in range(1,101):
    tournament_logs.append(
        tsv_to_df("tournament", n)
    )
    elexicase_logs.append(
        tsv_to_df("e_lexicase", n)
    )
    

In [16]:
# print all individual logs

for idx, (a, b) in enumerate(zip(tournament_logs, elexicase_logs)):
    print(f"{idx+1}.th Run:\nTournament-Selection:\n{a.head()}\nE-Lexicase-Selection:\n{b.head()}\n--------------\n")

1.th Run:
Tournament-Selection:
   gen  nevals  mean_training_se  std_training_se  min_training_se  \
0    0     500       937393000.0     1.978350e+10          40.8756   
1    1     113          101204.0     1.347520e+06          40.8756   
2    2     105       436909000.0     9.758550e+09          40.8756   
3    3     118           72054.7     7.129410e+05          40.8756   
4    4     108       437002000.0     9.759480e+09          40.8756   

   max_training_se  elite_testing_mse  elite_testing_err_std  
0     4.425280e+11                NaN                    NaN  
1     2.422550e+07             32.796                42.3609  
2     2.184260e+11             32.796                42.3609  
3     1.118210e+07             32.796                42.3609  
4     2.184470e+11             32.796                42.3609  
E-Lexicase-Selection:
   gen  nevals  mean_training_se  std_training_se  min_training_se  \
0    0     500       888982000.0     1.733020e+10          73.7153   
1    1 

In [17]:
def to_master_record(dfs: List[pd.DataFrame]) -> pd.DataFrame:
    
    """
    Summarize and return the results from each individual dataframe into a master record
    """
    
    headers = dfs[0].columns.values.tolist()
    ngens = len(dfs[0]["gen"])
    
    master = pd.DataFrame(0, index=np.arange(ngens), columns=headers)
    
    def mean_stddev(std_devs: List[float]) -> float:
        """returns the mean for a list of std_deviations """
        agg = 0.0
        for std_dev in std_devs:
            agg += std_dev ** 2
        return sqrt(agg / len(std_devs))
    
    for header in headers:
                
        for gen in range(ngens):
            
            vals = []
            
            for df in dfs:
                vals.append(
                    float(df[header].iloc[gen])
                )

            master.loc[gen,header]
            if not "std" in header:
                master.loc[gen,header] = mean(vals)
            
            else:
                master.loc[gen,header] = mean_stddev(vals)
                
    return master



master_tournmament = to_master_record(tournament_logs)
master_elexicase = to_master_record(elexicase_logs)
    

In [92]:
master_tournmament.to_csv(path_or_buf=f"{TABLE_PATH}/csv/master_tournament.csv")
master_tournmament.to_markdown(buf=f"{TABLE_PATH}/md/master_tournament.md")

master_tournmament

,gen,nevals,mean_training_se,std_training_se,min_training_se,max_training_se,elite_testing_mse,elite_testing_err_std
0,0,50.00,7.206743e+10,4.900325e+12,144.530728,3.601899e+12,NaN,NaN
1,1,12.32,5.027360e+07,3.198816e+09,135.263557,2.513529e+09,134.531064,165.720440
2,2,11.80,5.106664e+05,3.201457e+07,128.480600,2.490615e+07,128.053816,157.283951
3,3,11.38,1.146125e+07,6.829816e+08,118.798197,5.723577e+08,117.027291,142.951712
4,4,12.36,5.748104e+06,2.858449e+08,114.990169,2.869118e+08,113.473410,141.286382
5,5,12.03,4.578615e+07,3.139989e+09,111.902936,2.285297e+09,110.906479,136.094299


In [93]:
master_tournmament.describe().to_csv(path_or_buf=f"{TABLE_PATH}/csv/master_tournament_descriptive.csv")
master_tournmament.describe().to_markdown(buf=f"{TABLE_PATH}/md/master_tournament_descriptive.md")

master_tournmament.describe()

,gen,nevals,mean_training_se,std_training_se,min_training_se,max_training_se,elite_testing_mse,elite_testing_err_std
count,6.000000,6.000000,6.000000e+00,6.000000e+00,6.000000,6.000000e+00,5.000000,5.000000
mean,2.500000,18.315000,1.203020e+10,8.179441e+11,125.661031,6.012637e+11,120.798412,148.667356
std,1.870829,15.526635,2.941212e+10,1.999951e+12,12.700196,1.470006e+12,10.086550,12.351948
min,0.000000,11.380000,5.106664e+05,3.201457e+07,111.902936,2.490615e+07,110.906479,136.094299
25%,1.250000,11.857500,7.176390e+06,3.851291e+08,115.942176,3.582733e+08,113.473410,141.286382
50%,2.500000,12.175000,2.862370e+07,1.911485e+09,123.639398,1.428827e+09,117.027291,142.951712
75%,3.750000,12.350000,4.915174e+07,3.184109e+09,133.567818,2.456471e+09,128.053816,157.283951
max,5.000000,50.000000,7.206743e+10,4.900325e+12,144.530728,3.601899e+12,134.531064,165.720440


In [74]:
master_elexicase.to_csv(path_or_buf=f"{TABLE_PATH}/csv/master_elexicase.csv")
master_elexicase.to_markdown(buf=f"{TABLE_PATH}/md/master_elexicase.md")

master_elexicase

,gen,nevals,mean_training_se,std__training_se,min__training_se,max_training_se,elite_testing_mse,elite_testing_err_std
0,0,50.00,1.535816e+12,8.118274e+13,160.231312,7.678340e+13,NaN,NaN
1,1,11.76,8.611991e+06,3.232122e+08,139.814074,3.149322e+08,140.024299,173.735264
2,2,12.05,4.821770e+06,3.341364e+08,127.136589,2.409695e+08,128.119909,155.995326
3,3,11.83,2.273950e+06,1.548741e+08,124.083187,1.134748e+08,125.176740,152.825805
4,4,11.91,3.432598e+06,2.393332e+08,118.576303,1.714633e+08,120.359152,148.123950
5,5,12.12,9.497630e+06,6.634831e+08,116.390129,4.747238e+08,118.177356,144.859584


In [75]:
master_elexicase.describe().to_csv(path_or_buf=f"{TABLE_PATH}/csv/master_elexicase_descriptive.csv")
master_elexicase.describe().to_markdown(buf=f"{TABLE_PATH}/md/master_elexicase_descriptive.md")

master_elexicase.describe()

,gen,nevals,mean_training_se,std__training_se,min__training_se,max_training_se,elite_testing_mse,elite_testing_err_std
count,6.000000,6.000000,6.000000e+00,6.000000e+00,6.000000,6.000000e+00,5.000000,5.000000
mean,2.500000,18.278333,2.559741e+11,1.353074e+13,131.038599,1.279745e+13,126.371491,155.107986
std,1.870829,15.540957,6.269919e+11,3.314258e+13,16.508867,3.134658e+13,8.575774,11.255862
min,0.000000,11.760000,2.273950e+06,1.548741e+08,116.390129,1.134748e+08,118.177356,144.859584
25%,1.250000,11.850000,3.779891e+06,2.603030e+08,119.953024,1.888398e+08,120.359152,148.123950
50%,2.500000,11.980000,6.716880e+06,3.286743e+08,125.609888,2.779508e+08,125.176740,152.825805
75%,3.750000,12.102500,9.276220e+06,5.811464e+08,136.644703,4.347759e+08,128.119909,155.995326
max,5.000000,50.000000,1.535816e+12,8.118274e+13,160.231312,7.678340e+13,140.024299,173.735264


In [143]:
def aggregate_cells(dfs: List[pd.DataFrame], header, row) -> List[float]:
    
    vals = []

    for df in dfs:
        
        vals.append(
            df[header].iloc[row]
        )
    
    return vals

LAST_ROW = len(tournament_logs[0]) - 1

# aggregate training errors for elite models in last generation
tournament_elite_training_errors = aggregate_cells(tournament_logs, "min_training_se", LAST_ROW)
elexicase_elite_training_errors = aggregate_cells(elexicase_logs, "min_training_se", LAST_ROW)

# aggregate elite model performance on testing data
tournament_elite_testing_errors = aggregate_cells(tournament_logs, "elite_testing_mse", LAST_ROW)
elexicase_elite_testing_errors = aggregate_cells(elexicase_logs, "elite_testing_mse", LAST_ROW)

In [156]:
# Check if data series are normal distributed at alpha=5%

def is_normal_distr(vals: List[float], alpha:float=0.05) -> bool:
    
    print("Null Hypothesis: Sample comes from a normal distribution")
    
    statistic, pval = normaltest(vals)
    
    print(f"Statistic: {statistic}\nP-Value: {pval}")
    
    if pval < alpha:
        print("The null hypothesis can be rejected")
    else:
        print("The null hypothesis cannot be rejected")
    print("----------")
        

is_normal_distr(tournament_elite_training_errors)
is_normal_distr(elexicase_elite_training_errors)
is_normal_distr(tournament_elite_testing_errors)
is_normal_distr(elexicase_elite_testing_errors)


Null Hypothesis: Sample comes from a normal distribution
Statistic: 25.16665646744599
P-Value: 3.4287039212410135e-06
The null hypothesis can be rejected
----------
Null Hypothesis: Sample comes from a normal distribution
Statistic: 58.01615012112525
P-Value: 2.523208102344635e-13
The null hypothesis can be rejected
----------
Null Hypothesis: Sample comes from a normal distribution
Statistic: 29.68846549458086
P-Value: 3.5746343945868624e-07
The null hypothesis can be rejected
----------
Null Hypothesis: Sample comes from a normal distribution
Statistic: 57.71422868368015
P-Value: 2.934367322710921e-13
The null hypothesis can be rejected
----------


In [157]:
def test_mannwhitneyu(df: pd.DataFrame, xheader: str, yheader: str) -> Tuple[float,float]:
    
    ALPHA = 0.05

    statistic, pval = mannwhitneyu(x = df[xheader],y = df[yheader])
    print(f"Statistic: {statistic}\nPVal: {pval}\nPVal < ALPHA: {pval < ALPHA}")

    if pval > ALPHA:
        print(f"Results supports H0 for alpha={ALPHA}\n H0: The distribution underlying sample {xheader} is the same as the distribution underlying sample {yheader}")

    else:
        print(f"H0 can be rejected for alpha={ALPHA}\nThe distribution underlying sample {xheader} is NOT the same as the distribution underlying sample {yheader}")
    
    return statistic, pval


In [ ]:
fit_train_square_mwu = test_mannwhitneyu(results_square, "fit_train_tournament", "fit_train_elexicase")

plt.figure()

subplot_train_squared = results_square.boxplot(column=[HEADERS[0], HEADERS[2]])
subplot_train_squared.set_ylabel("MSE")
subplot_train_squared.set_title(f"Distribution for n={n_squared} runs")

plt.savefig("../docs/rmd/plots/box_train_square.png")

In [ ]:
fit_train_abs_mwu = test_mannwhitneyu(results_absolute, "fit_train_tournament", "fit_train_elexicase")

plt.figure()

subplot_train_absolute = results_absolute.boxplot(column=[HEADERS[0], HEADERS[2]])
subplot_train_absolute.set_ylabel("MAE")
subplot_train_absolute.set_title(f"Distribution for n={n_absolute} runs")

plt.savefig("../docs/rmd/plots/box_train_absolute.png")

In [ ]:
fit_test_square_mwu = test_mannwhitneyu(results_square, "fit_test_tournament", "fit_test_elexicase")

plt.figure()

subplot_test_squared = results_square.boxplot(column=[HEADERS[1], HEADERS[3]])
subplot_test_squared.set_ylabel("MSE")
subplot_test_squared.set_title(f"Distribution for n={n_squared} runs")

plt.savefig("../docs/rmd/plots/box_test_square.png")


In [ ]:
fit_test_abs_mwu = test_mannwhitneyu(results_absolute, "fit_test_tournament", "fit_test_elexicase")

plt.figure()

subplot_test_absolute = results_absolute.boxplot(column=[HEADERS[1], HEADERS[3]])
subplot_test_absolute.set_ylabel("MAE")
subplot_test_absolute.set_title(f"Distribution for n={n_absolute} runs")

plt.savefig("../docs/rmd/plots/box_test_abs.png")



In [ ]:
# save mwu results in csv files

ALPHA = 0.05
PATH = "../docs/rmd/tables/testing_mwu.csv"

with open(PATH, "w") as file:
    file.write("phase,fitness_function,statistic,p-val,lt_alpha\n")
    file.write(f"testing,mse,{fit_test_square_mwu[0]},{fit_test_square_mwu[1]},{fit_test_square_mwu[1] < ALPHA}\n")
    file.write(f"testing,mae,{fit_test_abs_mwu[0]},{fit_test_abs_mwu[1]},{fit_test_abs_mwu[1] < ALPHA}\n")

PATH = "../docs/rmd/tables/training_mwu.csv"

with open(PATH, "w") as file:
    file.write("phase,fitness_function,statistic,p-val,lt_alpha\n")
    file.write(f"training,mse,{fit_train_square_mwu[0]},{fit_train_square_mwu[1]},{fit_train_square_mwu[1] < ALPHA}\n")
    file.write(f"training,mae,{fit_train_abs_mwu[0]},{fit_train_abs_mwu[1]},{fit_train_abs_mwu[1] < ALPHA}\n")

    